In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from my_envs import *
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [3]:
import glob

In [48]:
env = MyEnvpool('Pong-v5', num_envs=8)
env.reset();

In [49]:
obs, rew, term, trunc, info = env.step(env.action_space.sample())

In [50]:
print(obs.shape)
print(info['env_id'])

(8, 4, 84, 84)
[0 1 2 3 4 5 6 7]


In [51]:
env.env.send(env.action_space.sample())

In [52]:
obs, rew, term, trunc, info = env.env.recv()

In [53]:
print(obs.shape)
print(info['env_id'])

(8, 4, 84, 84)
[0 1 2 3 4 5 6 7]


In [3]:
with open('atari_games_57.txt') as f:
    env_ids = f.read().split('\n')
env_ids = env_ids[:49]
env_ids = [env_id for env_id in env_ids if env_id!='Skiing']

In [4]:
env = []
for env_id in env_ids:
    envi = MyEnvpool(f"{env_id}-v5", num_envs=8, stack_num=1, episodic_life=True, reward_clip=True, seed=0, full_action_space=True)
    env.append(envi)
env = ConcatEnv(env)
env.reset();

In [6]:
%%time
for i in tqdm(range(1000)):
    env.step(env.action_space.sample())

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 6min 49s, sys: 2.61 s, total: 6min 52s
Wall time: 1min 15s


In [37]:
%%time
for i in tqdm(range(1000)):
    for envi in env.envs:
        envi.env.send(envi.action_space.sample())
        envi.env.recv()

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 6min 47s, sys: 2.17 s, total: 6min 49s
Wall time: 1min 14s


In [38]:
%%time
for i in tqdm(range(1000)):
    for envi in env.envs:
        envi.env.send(envi.action_space.sample())
    for envi in env.envs:
        envi.env.recv()

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 9min 47s, sys: 5.53 s, total: 9min 53s
Wall time: 21.9 s


In [5]:
%%time
for i in tqdm(range(1000)):
    env.step_async(env.action_space.sample())

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 9min 56s, sys: 5.91 s, total: 10min 2s
Wall time: 22.3 s


In [13]:
archive = np.load('../atari/data/ge_specialist/ge_specialist_MontezumaRevenge_0000.npy', allow_pickle=True).item()

In [31]:
class GEBuffer(Buffer):
    def __init__(self, env, n_steps, sample_traj_fn, device=None):
        # TODO mange devices
        super().__init__(env, n_steps, device="cpu")
        self.trajs = [None for _ in range(self.env.num_envs)]
        self.traj_lens = np.zeros(self.env.num_envs, dtype=int)
        self.i_locs = np.zeros(self.env.num_envs, dtype=int)
        self.sample_traj_fn = sample_traj_fn

    def reset_with_newtraj(self, ids):
        for id in ids:
            self.trajs[id] = self.sample_traj_fn(id)
            self.traj_lens[id] = len(self.trajs[id])
            self.i_locs[id] = 0
        obs, info = self.env.reset_subenvs(ids, seed=[0 for _ in ids])
        # assert np.array_equal(self.env.envs[id].ale.getRAM(), self.ram_start), "env reset to seed=0"
        return obs

    def gecollect(self, pbar=None):
        if self.first_collect:
            self.first_collect = False
            self.obs = self.reset_with_newtraj(range(self.env.num_envs))
        for t in range(self.n_steps):
            self.data["obs"][:, t] = self.obs
            action = np.array([traj[i_loc] for traj, i_loc in zip(self.trajs, self.i_locs)])
            self.i_locs += 1
            self.data["obs"][:, t] = action
            self.obs, _, term, trunc, _ = self.env.step(action)
            assert not any(term) and not any(trunc), "found a done in the ge buffer"

            ids_reset = np.where(self.i_locs >= self.traj_lens)[0]
            if len(ids_reset) > 0:
                self.obs[ids_reset] = self.reset_with_newtraj(ids_reset)
            if pbar is not None:
                pbar.update(1)

NameError: name 'Buffer' is not defined

In [39]:
def load_env_id2archives(env_ids, ge_data_dir, n_archives):
    env_id2archives = {}
    for env_id in tqdm(env_ids):
        files = sorted(glob.glob(f"{ge_data_dir}/*{env_id}*"))
        files = files[: n_archives]
        env_id2archives[env_id] = [np.load(f, allow_pickle=True).item() for f in files]
    return env_id2archives

def get_env_id2trajs(env_id2archives, strategy="best"):
    env_id2trajs = {}
    for env_id, archives in tqdm(env_id2archives.items()):
        env_id2trajs[env_id] = []
        for archive in archives:
            trajs, rets, novelty, is_leaf = archive["traj"], archive["ret"], archive["novelty"], archive["is_leaf"]
            if strategy == "all":
                idx = np.arange(len(trajs))
            elif strategy == "best":
                idx = np.array([np.argmax(rets)])
            elif strategy == "leaf":
                idx = np.nonzero(is_leaf)[0]
            else:
                raise ValueError(f"Unknown strategy: {strategy}")
            trajs = trajs[idx]
            env_id2trajs[env_id].extend(trajs)
    return env_id2trajs


env_id2archives = load_env_id2archives(['MontezumaRevenge', 'Pitfall'], '../atari/data/ge_specialist/', 30)
env_id2trajs = get_env_id2trajs(env_id2archives, strategy='best')


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
env_ids = ['MontezumaRevenge', 'Pitfall']
env_id2archives = load_env_id2archives(env_ids, '../atari/data/ge_specialist/', 30)
env_id2trajs = get_env_id2trajs(env_id2archives, strategy='best')

for env_id, trajs in env_id2trajs.items():
    print(f"env_id: {env_id}, #trajs: {len(trajs)}")

envs = []
for env_id in args.env_ids:
    envi = MyEnvpool(f"{env_id}-v5", num_envs=args.n_envs, stack_num=1, episodic_life=True, reward_clip=True, seed=args.seed, full_action_space=True)
    if args.norm_rew:
        envi = gym.wrappers.NormalizeReward(envi, gamma=args.gamma)
    envi = RecordEpisodeStatistics(envi, deque_size=32)
    envi = ToTensor(envi, device=args.device)
    envs.append(envi)
env = ConcatEnv(envs)

# make_env_fns = []
# for env_id in args.env_ids:
#     make_env_fns.extend([partial(make_ge_env_single, env_id=env_id) for _ in range(args.n_envs)])
# env = MyAsyncVectorEnv(make_env_fns)
# print("Done creating envs!")

# def sample_traj_fn(id):
#     env_id = args.env_ids[id // args.n_envs]
#     trajs = env_id2trajs[env_id]
#     return trajs[np.random.choice(len(trajs))]

# print("Creating buffer")
# buf = GEBuffer(env, args.n_steps, sample_traj_fn=sample_traj_fn, device=args.device)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

env_id: MontezumaRevenge, #trajs: 30
env_id: Pitfall, #trajs: 30


In [16]:
with open('atari_games_57.txt') as f:
    env_ids = f.read().split('\n')

In [17]:
for env_id in env_ids:
    print(env_id)
    env = MyEnvpool(f'{env_id}-v5', num_envs=8, stack_num=1, frame_skip=4, repeat_action_probability=0.0, noop_max=1, use_fire_reset=False, full_action_space=True)
    o = np.stack([env.reset()[0] for i in range(20)])
    assert (o.var(axis=(0, 1))==0).all()

Alien
Amidar
Assault
Asterix
Asteroids
Atlantis
BankHeist
BattleZone
BeamRider
Berzerk
Bowling
Boxing
Breakout
Centipede
ChopperCommand
CrazyClimber
Defender
DemonAttack


KeyboardInterrupt: 

In [51]:
logits = torch.randn(128, 32, 18)
logits2 = torch.randn(128, 32, 18)
d = torch.distributions.Categorical(logits=logits)
d2 = torch.distributions.Categorical(logits=logits2)
logits, logits2 = d.logits, d2.logits

a = torch.nn.functional.kl_div(logits, logits2, log_target=True, reduction='none').sum(dim=-1)
b = torch.distributions.kl_divergence(d2, d)
assert torch.allclose(a, b)
print(a.shape, b.shape)

torch.Size([128, 32]) torch.Size([128, 32])


In [54]:
logits = torch.randn(128, 32, 18)
target = torch.randint(0, 18, (128, 32))
d = torch.distributions.Categorical(logits=logits)
logits = d.logits

a = torch.nn.functional.cross_entropy(logits, target, reduction='none')
print(a.shape)

RuntimeError: Expected target size [128, 18], got [128, 32]

In [4]:
from ge_bc import *

In [88]:
args = '--strategy=best --device=cuda --n_envs=4 --n_steps=256 --batch_size=384 --ge_data_dir=../atari/data/ge_specialist/'
args += ' --env_ids Amidar Assault Asteroids Atlantis BankHeist BattleZone Bowling Boxing Breakout Centipede ChopperCommand CrazyClimber Defender DemonAttack DoubleDunk Enduro FishingDerby Freeway Frostbite Gopher Gravitar Hero IceHockey Kangaroo Krull KungFuMaster MontezumaRevenge MsPacman NameThisGame Phoenix Pitfall Pong PrivateEye Riverraid RoadRunner Robotank Seaquest Solaris SpaceInvaders StarGunner Surround Tennis TimePilot Tutankham UpNDown Venture VideoPinball YarsRevenge'
args = parse_args(args.split(' '))




In [89]:
args

Namespace(track=False, entity=None, project=None, name=None, device='cuda', seed=0, model='cnn_4', load_ckpt=None, save_ckpt=None, n_ckpts=1, lr=0.00025, clip_grad_norm=1.0, env_ids=['Amidar', 'Assault', 'Asteroids', 'Atlantis', 'BankHeist', 'BattleZone', 'Bowling', 'Boxing', 'Breakout', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'Defender', 'DemonAttack', 'DoubleDunk', 'Enduro', 'FishingDerby', 'Freeway', 'Frostbite', 'Gopher', 'Gravitar', 'Hero', 'IceHockey', 'Kangaroo', 'Krull', 'KungFuMaster', 'MontezumaRevenge', 'MsPacman', 'NameThisGame', 'Phoenix', 'Pitfall', 'Pong', 'PrivateEye', 'Riverraid', 'RoadRunner', 'Robotank', 'Seaquest', 'Solaris', 'SpaceInvaders', 'StarGunner', 'Surround', 'Tennis', 'TimePilot', 'Tutankham', 'UpNDown', 'Venture', 'VideoPinball', 'YarsRevenge'], n_iters=10000, n_envs=4, n_steps=256, batch_size=384, n_updates=16, ent_coef=0.0, ge_data_dir='../atari/data/ge_specialist/', strategy='best', n_archives=1, min_traj_len=100)

In [90]:
env_teacher = make_env(args)

print("Loading archives")
env_id2archives = load_env_id2archives(args.env_ids, args.ge_data_dir, args.n_archives)
print("Creating trajs")
env_id2trajs = get_env_id2trajs(env_id2archives, strategy=args.strategy, min_traj_len=args.min_traj_len)
for env_id, trajs in env_id2trajs.items():
    print(f"env_id: {env_id}, #trajs: {len(trajs)}")

def sample_traj_fn(id):
    env_id = args.env_ids[id // args.n_envs]
    trajs = env_id2trajs[env_id]
    return trajs[np.random.choice(len(trajs))]

print("Creating buffer...")
buffer_teacher = GEBuffer(env_teacher, args.n_steps, sample_traj_fn=sample_traj_fn, device=args.device)

Loading archives


  0%|          | 0/48 [00:00<?, ?it/s]

Creating trajs


  0%|          | 0/48 [00:00<?, ?it/s]

env_id: Amidar, #trajs: 1
env_id: Assault, #trajs: 1
env_id: Asteroids, #trajs: 1
env_id: Atlantis, #trajs: 1
env_id: BankHeist, #trajs: 1
env_id: BattleZone, #trajs: 1
env_id: Bowling, #trajs: 1
env_id: Boxing, #trajs: 1
env_id: Breakout, #trajs: 1
env_id: Centipede, #trajs: 1
env_id: ChopperCommand, #trajs: 1
env_id: CrazyClimber, #trajs: 1
env_id: Defender, #trajs: 1
env_id: DemonAttack, #trajs: 1
env_id: DoubleDunk, #trajs: 1
env_id: Enduro, #trajs: 1
env_id: FishingDerby, #trajs: 1
env_id: Freeway, #trajs: 1
env_id: Frostbite, #trajs: 1
env_id: Gopher, #trajs: 1
env_id: Gravitar, #trajs: 1
env_id: Hero, #trajs: 1
env_id: IceHockey, #trajs: 1
env_id: Kangaroo, #trajs: 1
env_id: Krull, #trajs: 1
env_id: KungFuMaster, #trajs: 1
env_id: MontezumaRevenge, #trajs: 1
env_id: MsPacman, #trajs: 1
env_id: NameThisGame, #trajs: 1
env_id: Phoenix, #trajs: 1
env_id: Pitfall, #trajs: 1
env_id: Pong, #trajs: 1
env_id: PrivateEye, #trajs: 1
env_id: Riverraid, #trajs: 1
env_id: RoadRunner, #trajs:

In [91]:
print("Warming up buffer...")
ds = []
for i_iter in tqdm(range(30)):
    buffer_teacher.collect()
    for i in range(10):
        obs1 = buffer_teacher.generate_batch(args.batch_size, 1)['obs']
        obs2 = buffer_teacher.generate_batch(args.batch_size, 1)['obs']
        obs1 = repeat(rearrange((obs1-128.)/255., 'b 1 1 h w -> b 1 h w'), 'b 1 h w -> b 3 h w')
        obs2 = repeat(rearrange((obs2-128.)/255., 'b 1 1 h w -> b 1 h w'), 'b 1 h w -> b 3 h w')
        d = loss_fn_alex(obs1, obs2)
        ds.append(d.mean().item())

Warming up buffer...


  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [87]:
np.mean(ds)

0.1031223738193512

In [64]:
from einops import rearrange, repeat

In [41]:
import lpips
loss_fn_alex = lpips.LPIPS(net='alex').to(args.device) # best forward scores

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /data/vision/phillipi/akumar01/miniconda3/envs/egb2/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


In [44]:
import torch
img0 = torch.zeros(10,3,64,64).to(args.device) # image should be RGB, IMPORTANT: normalized to [-1,1]
img1 = torch.zeros(10,3,64,64).to(args.device)
d = loss_fn_alex(img0, img1)

In [46]:
d.shape

torch.Size([10, 1, 1, 1])